In [2]:
from skimage import color
from skimage import io
import pathlib
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose

from typing import Any, Callable, Sequence, Optional

import pathlib
import os

import pandas as pd
from torchvision.io import read_image

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")



# This gets the path as a variable
# MAKE SURE YOU ARE IN SCRATCH OR PSCRATCH
current_dir = pathlib.Path().resolve()
print(current_dir)
pd_train_path = current_dir / 'Data' / 'BSDS300' / 'train' 
pd_test_path = current_dir / 'Data' / 'BSDS300' / 'test'
assert pd_train_path.exists()
assert pd_test_path.exists()


ModuleNotFoundError: No module named 'torch'

In [ ]:
import glob
# but glob.glob didn't register the path var, so had to type it all out
train_filelist = glob.glob(str(pd_train_path / '*.jpg'))
test_filelist = glob.glob(str(pd_test_path /'*.jpg'))

clean_train_img = tf.io.read_file(train_filelist[0])


In [ ]:
@tf.function
def load_image(filename):
    raw = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(raw, channels=3)
    # the `print` executes during tracing.
    print("Initial shape: ", image.shape)
    image.set_shape([28, 28, 3])
    print("Final shape: ", image.shape)
    return image

In [ ]:
load_image(train_filelist[0])

This code was found [here](https://stackoverflow.com/questions/39195113/how-to-load-multiple-images-in-a-numpy-array)

In [ ]:
import glob
# but glob.glob didn't register the path var, so had to type it all out
train_filelist = glob.glob('/pscratch/sd/m/mdowicz/DESI_dn/Data/BSDS300/train/*.jpg')
test_filelist = glob.glob('/pscratch/sd/m/mdowicz/DESI_dn/Data/BSDS300/test/*.jpg')

# This creates a numpy array of the images but has shape (num_samp,) and then [0].shape = (width, height, channels)
pd_train = np.array([np.array(Image.open(fname)) for fname in train_filelist], dtype=object)
pd_test = np.array([np.array(Image.open(fname)) for fname in test_filelist], dtype=object)

# pd_train_list = glob.glob(pd_train_path / '*.jpg')
# pd_test_list = glob.glob(pd_test_path / '*.jpg')

# pd_train = np.array([np.array(Image.open(fname)) for fname in pd_train_list])
# pd_test = np.array([np.array(Image.open(fname)) for fname in pd_test_list])


In [ ]:
pd_train[0].shape

In [ ]:
idx = 9
plt.imshow(pd_test[idx], interpolation='none')
plt.axis('off')
print(pd_test[idx].shape)

In [ ]:
class Paper_Dataset(object):
    def __init__(self, images_dir, patch_size, gaussian_noise_level, seed=1234):
        # self.image_files = sorted(glob.glob(images_dir + '/*'))
        self.image_files = sorted(glob.glob(images_dir))
        self.patch_size = patch_size
        self.gaussian_noise_level = gaussian_noise_level
        self.seed = seed

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        clean_image = tf.io.read_file(self.image_files[idx])
        clean_image = tf.image.decode_jpeg(clean_image, channel=3)
        clean_image = pil_image.fromarray(clean_image.numpy())
        
        # randomly crop patch from training set
        crop_x = random.randint(0, clean_image.width - self.patch_size)
        crop_y = random.randint(0, clean_image.height - self.patch_size)
        clean_image = clean_image.crop((crop_x, crop_y, crop_x + self.patch_size, crop_y + self.patch_size))
        
        noisy_image = clean_image.copy()
        gaussian_noise = np.zeros((clean_image.height, clean_image.width, 3), dtype=np.float32)
        
        # additive gaussian noise
        if self.gaussian_noise_level is not None:
            if len(self.gaussian_noise_level) == 1:
                sigma = self.gaussian_noise_level[0]
            else:
                sigma = random.randint(self.gaussian_noise_level[0], self.gaussian_noise_level[1])
            gaussian_noise += np.random.normal(0.0, sigma, (clean_image.height, clean_image.width, 3)).astype(np.float32)
            
            
        clean_image = np.array(clean_image).astype(np.float32)
        noisy_image = np.array(noisy_image).astype(np.float32)
        noisy_image += gaussian_noise
        
        
        d_input = np.transpose(noisy_image, axes=[2, 0, 1])
        labels = np.transpose(clean_image, axes=[2, 0, 1])
        
        
        # normalization
        d_input /= 255.0
        label /= 255.0
        
        return d_input, label
        

In [3]:
train_paper = Paper_Dataset(str(pd_train_path), patch_size=50, gaussian_noise_level = [0, 55])

NameError: name 'Paper_Dataset' is not defined

In [4]:
p_train_dataloader = DataLoader(train_paper, batch_size=64, shuffle=True)


NameError: name 'DataLoader' is not defined

In [ ]:
img_dataset = Img_Dataset(training_data, patch_size=50)
train_dataloader = DataLoader(img_dataset, batch_size=64, shuffle=True)

train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

In [ ]:
img = train_features[0][0].squeeze()
imgs = train_features.cpu().detach().numpy()
plt.imshow(imgs[0], cmap="gray")

In [ ]:
def pair_show(dataloader, igen):
    """
    Plot to compare a training sample and its residual noise image
    
    Parameters
    ----------
    igen : int
           Index of specific sample of interest
    """
    
    train_features, train_labels = next(iter(train_dataloader))
    train_features = train_features.cpu().detach().numpy()
    train_labels = train_labels.cpu().detach().numpy()
    fig, ax = plt.subplots(1, 2, figsize=(8,4))
    ax[0].imshow(train_features[igen], origin='lower', interpolation='none')
    ax[0].axis('off')
    ax[0].set_title('Noisy Sample')
    ax[1].imshow(train_labels[igen], origin='lower', interpolation='none')
    ax[1].axis('off')
    ax[1].set_title('Clean Sample')

In [ ]:
pair_show(train_dataloader, 10)

# Old code

This code was found [here](https://stackoverflow.com/questions/26392336/importing-images-from-a-directory-python-to-list-or-dictionary)

In [ ]:
# # This loads a directory of images

# def load_jpg_dir(path):
#     imgs = []
#     path = path
#     valid_images = [".jpg", ".gif", ".png", ".tga"]
#     for f in os.listdir(path):
#         ext = os.path.splitext(f)[1]
#         if ext.lower() not in valid_images:
#             continue
#         imgs.append(Image.open(os.path.join(path,f)))
        
#     for i in range(len(imgs)):
        
        
        
#     return imgs

Code below was found somewhere else, but lost the tab

In [ ]:
# from PIL import Image
# import numpy as np


# # This loads a file into np, but its eh
# # The first bit of code works well enough....

# def load_image( infilename ) :
#     img = Image.open( infilename )
#     img.load()
#     data = np.asarray( img, dtype="int32" )
#     return data

# def save_image( npdata, outfilename ) :
#     img = Image.fromarray( np.asarray( np.clip(npdata,0,255), dtype="uint8"), "L" )
#     img.save( outfilename )

In [ ]:
# pd_train = load_jpg_dir(pd_train_path)
# pd_test = load_jpg_dir(pd_test_path)

# q = np.asarray(pd_train[0].load(), dtype="int32")
# # pd_train = np.asarray(pd_train, dtype=np.float32)
# # pd_test = np.asarray(pd_test)